In [105]:
from pyspark.sql import SparkSession

In [106]:
spark = SparkSession.builder.appName('cluster').getOrCreate()

In [107]:
dataset = spark.read.csv('seeds_dataset.csv', inferSchema=True, header=True)

In [108]:
dataset.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)



In [109]:
dataset.head(1)

[Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22)]

In [110]:
from pyspark.ml.clustering import KMeans

In [111]:
from pyspark.ml.feature import VectorAssembler

In [112]:
dataset.columns

['area',
 'perimeter',
 'compactness',
 'length_of_kernel',
 'width_of_kernel',
 'asymmetry_coefficient',
 'length_of_groove']

In [113]:
assembler = VectorAssembler(inputCols=dataset.columns, outputCol='features')

In [114]:
final_data = assembler.transform(dataset)

In [115]:
final_data.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)
 |-- features: vector (nullable = true)



In [116]:
from pyspark.ml.feature import StandardScaler

In [117]:
scaler = (StandardScaler(inputCol='features', outputCol='scaledFeatures'))

In [118]:
scaler_model = scaler.fit(final_data)

In [119]:
final_data = scaler_model.transform(final_data)

In [120]:
final_data.head(1)

[Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22, features=DenseVector([15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22]), scaledFeatures=DenseVector([5.2445, 11.3633, 36.8608, 13.0072, 8.7685, 1.4772, 10.621]))]

In [121]:
kmeans = KMeans(featuresCol='scaledFeatures', k=3)

In [122]:
model = kmeans.fit(final_data)

In [123]:
print(model.summary.trainingCost)

429.0286883592525


In [124]:
centers = model.clusterCenters()

In [125]:
print(centers)

[array([ 4.06133795, 10.13721767, 35.82681204, 11.81771972,  7.5087187 ,
        3.25852121, 10.4215732 ]), array([ 4.88329439, 10.89351922, 37.27768719, 12.35125928,  8.56438491,
        1.80870857, 10.32847336]), array([ 6.31670546, 12.37109759, 37.39491396, 13.91155062,  9.748067  ,
        2.39849968, 12.2661748 ])]


In [126]:
model.transform(final_data).select('prediction').show()

+----------+
|prediction|
+----------+
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         2|
|         2|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         0|
+----------+
only showing top 20 rows

